<a href="https://colab.research.google.com/github/daryllman/walmart-scraper/blob/master/scraper_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load in the necessary libraries

In [1]:
import requests
from bs4 import BeautifulSoup as bs #pip install beautifulsoup4

## Load sample webpage content



In [5]:
# Load the sample webpage
#base_url = 'https://www.walgreens.com/store/store/category/productlist.jsp?N=359451&Eon=359451&No=98'
#base_url = 'https://www.walgreens.com/store/store/category/productlist.jsp?No=24&N=359451&Eon=359451' # normal
base_url = 'https://www.walgreens.com/store/store/category/productlist.jsp?No=3696&N=359451&Eon=359451' # with diff product card class names
r = requests.get(base_url)

# Convert to a beautiful soup object
soup = bs(r.content)

In [ ]:
#print(r.content)
#print(soup)
print(soup.prettify())

## Extract Item URLs from page


In [16]:
# Extract Item divs from page

item_divs = soup.findAll('div', id=lambda x: x and x.startswith('productcardprod'))
#item_row_divs = soup.findAll('div', id=lambda x: x and x.startswith('product-row-'))
#item_divs = [itemrowdiv.findChildren('div', id=re.compile("productcardprod"),recursive=False) for itemrowdiv in item_row_divs]
print(item_divs)
print(len(item_divs))

[<div class="col-xl-3 col-lg-3 col-md-6 col-sm-6 col-6 product-column pr0" data-reactid="900" id="productcardprod6310393"><div class="wag-product-cards wag-pos-rel" data-reactid="901"><div class="wag-prod-card-btm wag-pos-rel" data-reactid="902" id="productcardheightprod6310393"><div class="beagleprod" data-reactid="903"></div><div class="wag-product-card-details wag-product-card-width_b" data-reactid="904" id="productcard0"><div class="text-center mb20" data-reactid="905"><a data-reactid="906" href="/store/c/stacy's-pita-chips-parmesan-garlic/ID=prod6310393-product" id="productOmniSelectcompare_sku6262124" title="Stacy's Pita Chips Parmesan Garlic (7.33 oz )"><img alt="Stacy's Pita Chips Parmesan Garlic" class="img-responsive wag-familyoffer-productsimg imglazy " data-reactid="907" id="secImgcompare_sku6262124" src="//pics.drugstore.com/prodimg/567203/155.jpg"/><span class="sr-only" data-reactid="908"><!-- react-text: 909 -->Stacy's Pita Chips Parmesan Garlic<!-- /react-text --><!-- r

In [17]:
# Extract Item URL V1
import re
item_divs = soup.find_all('div', id=re.compile("productcardprod"))
url_links = [item.find('a', id=re.compile("productOmniSelectcompare")).get('href', '') for item in item_divs]
print(url_links)
print(len(url_links))

["/store/c/stacy's-pita-chips-parmesan-garlic/ID=prod6310393-product", '/store/c/franzia-chardonnay-wine/ID=prod6230704-product', '/store/c/sauza-silver-tequila/ID=prod6104868-product', '/store/c/budweiser-beer-black-crown,-25-ox-can/ID=prod6198647-product', '/store/c/sun-drop-soda-citrus/ID=prod6331885-product', '/store/c/frito-lay-potato-chips-sour-cream-and-onion/ID=prod6347938-product', "/store/c/campbell's-chunky-steak-%26-potato-soup/ID=prod6215269-product", '/store/c/russell-stover-big-bite-bar-caramel/ID=prod6252790-product']
8


In [25]:
# Extract Item URL V2
# After scraping, i found that some products are encapsulated under divs with class name 'productcard' only, hence are missed.
# A easy way for me to scrape, though not ideal is just to take all divs with the class name 'productcard' -- I will need to filter duplicates afterwards.
item_divs = soup.find_all('div', id=re.compile("productcard"))
url_links = [item.find('a', id=re.compile("productOmniSelectcompare")).get('href', '') for item in item_divs]
url_links = list(dict.fromkeys(url_links)) # to remove duplicates
print(url_links)
print(len(url_links))

["/store/c/stacy's-pita-chips-parmesan-garlic/ID=prod6310393-product", '/store/c/franzia-chardonnay-wine/ID=prod6230704-product', '/store/c/sauza-silver-tequila/ID=prod6104868-product', '/store/c/budweiser-beer-black-crown,-25-ox-can/ID=prod6198647-product', '/store/c/sun-drop-soda-citrus/ID=prod6331885-product', '/store/c/frito-lay-potato-chips-sour-cream-and-onion/ID=prod6347938-product', "/store/c/campbell's-chunky-steak-%26-potato-soup/ID=prod6215269-product", '/store/c/russell-stover-big-bite-bar-caramel/ID=prod6252790-product']
8


## Extract Item details from one item url

In [26]:
# Extract item detail from one product
base_url = "https://www.walgreens.com"
item1 = url_links[0] # for exploration purpose, we just take the first url from the list
item1_url = "{}{}".format(base_url,item1)
print(item1_url)



https://www.walgreens.com/store/c/stacy's-pita-chips-parmesan-garlic/ID=prod6310393-product


In [30]:
r = requests.get(item1_url)

# Convert to a beautiful soup object
item_soup = bs(r.content)

# Initialise json object for item
item_dict = dict()
item_dict["categoricalFeatures"] = {}
item_dict["textFeatures"] = {}
item_dict["numericalFeatures"] = {}


In [33]:
# Get Name
item_name = item_soup.find(id='productName').find('span').getText()
item_dict["textFeatures"]["name"] = item_name
item_name

'Pita Chips Parmesan Garlic'

In [34]:
# Get Image URL
item_img_url = item_soup.find(id='productImg').get('src')
item_dict["image_url"] = item_img_url
item_img_url

'https://pics.drugstore.com/prodimg/567203/900.jpg'

In [35]:
# Get Brand
item_brand = item_soup.find('a',attrs={'class':'brand-title'}).find('strong').getText()
item_dict["categoricalFeatures"]["brand"] = {"values":[item_brand]}
item_brand

"Stacy's"

In [37]:
# Get Categories
item_cat_list = item_soup.find('ul',attrs={'class':'nav__bread-crumbs'}).findAll('li')
item_cat_list_formatted = [cat.getText() for cat in item_cat_list]
remaining_list = item_cat_list_formatted[item_cat_list_formatted.index("Grocery"):]
item_dict["categoricalFeatures"]["category"] = {"values":remaining_list}
remaining_list

['Grocery', 'Snacks', 'Chips, Dips & Pretzels']

In [ ]:
# Get Price
# V1: - fails if there is no price..
price_list = item_soup.find('div',attrs={'id':'product'}).find('span',attrs={'class':'product__price'}).findAll()
price_list_formatted = [price.getText() for price in price_list]
price_string = "{}{}".format(price_list_formatted[0], '.'.join(price_list_formatted[1:]))
item_dict["numericalFeatures"]["price"] = price_string
price_list

In [42]:
# Get Price
# V2: - if no price, return empty string
price_list = item_soup.find('div', attrs={'id': 'product'}).find('span', attrs={'class': 'product__price'})
if price_list:
    price_list = price_list.findAll()
    price_list_formatted = [price.getText() for price in price_list]
    price_string = "{}{}".format(price_list_formatted[0], '.'.join(price_list_formatted[1:]))
else:
    price_string = ""
item_dict["numericalFeatures"]["price"] = price_string
price_string

''

In [46]:
# Get Rating
rating_val = item_soup.find('div', id="prodBV").find('span').get('aria-label').split()[0]
item_dict["numericalFeatures"]["rating"] = rating_val
rating_val

'no'

In [52]:
# Get Description
item_desc_list = item_soup.find('li', id="prodDesc").find('div',attrs={'class':'view-more-content__height'}).find_all(['li','p'])
item_desc_list_formatted = [desc.getText() for desc in item_desc_list]
item_desc_string = ', '.join(item_desc_list_formatted)
item_dict["textFeatures"]["description"] = item_desc_string
item_desc_string

''

In [51]:
# Get Ingredients
item_ingrd = item_soup.find('li', id="Ingredients").find('span').getText()
item_dict["textFeatures"]["ingredients"] = item_ingrd
item_ingrd

"Enriched Wheat Flour, (Wheat Flour, Niacin, Reduced Iron, Thiamin Mononitrate, Riboflavin, Folic Acid), or Canola Oil, Sunflower Oil and, (Pasteurized Cow's Milk, Sea Salt, Less Than 2% of the Following: Whole Wheat Flour, Organic Cane Sugar, Dried Garlic, Parmesan Cheese, (Part-Skim Milk, Cheese Cultures, Enzymes), Salt, Oat Fiber, Yeast, Parsley, Malted Barley Flour, Rosemary Extract, Ascorbic Acid"

In [53]:
# Get Nutrional Information
def format_nutri_servings_info(bs_div_item):
  full_str = bs_div_item.getText()
  key = bs_div_item.strong.getText().strip()
  val = full_str.lstrip(key).strip()
  return "{}: {}".format(key,val)

def format_nutri_body_info(bs_tr_item):
  intended_content = bs_tr_item.find('td')
  full_str = intended_content.getText()
  key = intended_content.strong.getText().strip()
  val = full_str.lstrip(key).strip()
  return "{}: {}".format(key,val)

item_nutri_list = []
item_nutri = item_soup.find('div',attrs={'class':'table-container nutrition'})
item_nutri_servings = item_nutri.find_all('div')
item_nutri_servings_formatted = [format_nutri_servings_info(item_s) for item_s in item_nutri_servings]
print(item_nutri_servings_formatted)

item_nutri_body = item_nutri.find('tbody').find_all('tr')
item_nutri_body_formatted = [format_nutri_body_info(item_s) for item_s in item_nutri_body]
print(item_nutri_body_formatted)

item_nutri_list = item_nutri_list + item_nutri_servings_formatted + item_nutri_body_formatted
print(item_nutri_list)

item_dict["textFeatures"]["nutritionInformation"] = {"values":item_nutri_list}

['Serving Size: 1 oz', 'Servings Per Container: About 7']
['Total Fat: 5g', 'Saturated Fat: 0.5g', 'Polyunsaturated fat: 0.5g', 'Monounsaturated fat: 4g', 'Cholesterol: 0mg', 'Sodium: 270mg', 'Total Carbohydrate: 18g', 'Dietary fiber: 1g', 'Sugars: Sugars <1g', 'Protein: 3g', 'Calories: 140', 'Calories From Fat - Calories: 50', 'Trans Fat - Total Fat: 0g', 'Vitamin A: 0units', 'Vitamin C: 0units', 'Calcium: 0units', 'Iron: 0units']
['Serving Size: 1 oz', 'Servings Per Container: About 7', 'Total Fat: 5g', 'Saturated Fat: 0.5g', 'Polyunsaturated fat: 0.5g', 'Monounsaturated fat: 4g', 'Cholesterol: 0mg', 'Sodium: 270mg', 'Total Carbohydrate: 18g', 'Dietary fiber: 1g', 'Sugars: Sugars <1g', 'Protein: 3g', 'Calories: 140', 'Calories From Fat - Calories: 50', 'Trans Fat - Total Fat: 0g', 'Vitamin A: 0units', 'Vitamin C: 0units', 'Calcium: 0units', 'Iron: 0units']


In [54]:
# Get Product ID
# UPC no. if available, if not product id
item_upc = item_soup.find('p',attrs={'class':'universal-Item-code'}).getText()
item_upc="".join(item_upc.replace(":","").split())
print(item_upc)
item_code = item_soup.find('p',attrs={'class':'universal-product-code'}).getText()
item_code = item_code.split()[-1]
print(item_code)

UPC02840056464
929146


In [55]:
import re
prod_id = item1_url.split('/')[-1]
prod_id_formatted = re.split('=|-',prod_id )[1]
prod_id_formatted

'prod6310393'

Further formatting of Description text


In [56]:
samplestr ="ted\u00a0& Approve.*\u00a0 Patented Fermentation Process.\u00a0 \"All-Natural\".\u00a0 Non-GMO., This"
#samplestr_formatted = samplestr.replace("\u00a0A", " ")
#samplestr_formatted = samplestr_formatted.replace("\\", " ")
samplestr_formatted = "".join(re.split('\u00a0A|..',samplestr ))
print(samplestr_formatted)

s
